In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from functools import reduce
from ast import literal_eval
from math import log

import pandas as pd
from numpy import mean
from nltk import word_tokenize
from nltk.corpus import stopwords
from IPython.display import Markdown, display, HTML

default_stopwords = set(stopwords.words('portuguese'))

In [3]:
data = pd.read_csv("../output/results.csv")
data.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


In [27]:
def split_spread_words(corpus, delim):
    """ Split then spread alpha word with certain delimiters.

    Split words with alphabetical characters that have certain 
    delimiters then spread the resulting words across the corpus.

    :param list corpus: list of words.
    :param str delim: target delimiter.

    :return: updated list of words 

    :rtype: list
    """
    
    new_words = []
    for word in corpus:
        if any(c.isalnum() for c in word):
            new_words.extend(word.split(delim))
        else:
            new_words.append(word)

    return new_words


def word_processing(text):
    """ Process text and returns list of valid terms.

    Process text, removes stopwords, remove invalid chars 
    and returns list of valid terms.
    
    :param str text: text to be processed.

    :return: text as valid list of words 

    :rtype: list(str)
    """
    
    words = word_tokenize(text)
    
    # Put everything in lower case
    words = [word.lower() for word in words]
    
    # Remove words that don't have at least one alphabetical character 
    words = [word for word in words if any(c.isalnum() for c in word)]

    # Remove hyphen at end of word
    words = [word[:-1] if word[-1] == '-' else word for word in words]

    # Remove hyphen at beggining of word
    words = [word[1:] if word[0] == '-' else word for word in words]

    # Split words joined by en dash
    words = [word for line in words for word in line.split('–')] 
    words = [word for line in words for word in line.split('—')] # different encoding 

    # Split words joined by dot if they are alphabetical
    words = split_spread_words(words, '.')

    # Remove lone punctuation from the splits
    words = [word for word in words if any(c.isalnum() for c in word)]

    # Remove stopwords
    words = [word for word in words if word.lower() not in default_stopwords]
    
    return words

In [28]:
data["sanitized_title"] = data["title"].apply(lambda text: word_processing(text))
data.head()

,title,subtitle,author,date,section,text,url,sanitized_title
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[sociedade, rubens, paiva, facínoras, mataram]"
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[justiça, suspende, decisão, proibia, forças, ..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[governo, bolsonaro, prega, negacionismo, hist..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[pais, gabo, perceberam, filho, mentiroso]"
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[rádios, canadenses, banem, músicas, michael, ..."


In [56]:
%%capture # Ignore gensim warnings 
from gensim.models import Word2Vec

model = Word2Vec.load('/home/benardi/workspace/bochica/data/pt/pt.bin')

In [38]:
def text_to_vectors(text, model):
    vectors = []
    for word in text:
        try:
            vectors.append(model.wv.get_vector(word))
        except:
            pass # Ignore unknown words

    return vectors

In [54]:
from math import inf
from math import sqrt

def word_mover_distance(vctrs1, vctrs2): 
    wmd = 0
    for vct1 in vctrs1:
        min_dist = inf
        for vct2 in vctrs2:
            min_dist = min(min_dist, sqrt(sum((vct1 - vct2) ** 2)))
  
        wmd += min_dist * 1.0/len(vctrs1)
 
    return wmd

In [55]:
word_mover_distance(text_to_vectors(data.xs(1)["sanitized_title"],model),
                    text_to_vectors(data.xs(2)["sanitized_title"],model))

19.57090659328106

In [59]:
data["title_asvectors"] = data["sanitized_title"].apply(lambda title: text_to_vectors(title, model)) 
data.head()

,title,subtitle,author,date,section,text,url,sanitized_title,title_asvectors
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...,"[sociedade, rubens, paiva, facínoras, mataram]","[[0.20829841, -3.2993062, -1.9657193, 2.004485..."
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...,"[justiça, suspende, decisão, proibia, forças, ...","[[-0.31644273, 0.5114689, -0.5958715, 1.125206..."
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...,"[governo, bolsonaro, prega, negacionismo, hist...","[[-1.3232566, -1.6326014, -2.5689228, 2.607748..."
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[pais, gabo, perceberam, filho, mentiroso]","[[-0.712125, 0.67016095, -0.59830016, -0.76418..."
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...,"[rádios, canadenses, banem, músicas, michael, ...","[[-1.2929834, -0.86154693, 1.2064326, 1.148315..."


In [86]:
def get_most_similar_news(news_title, news_df, model, n=3):

    vectrs1 = text_to_vectors(news_title,model)
    temp = news_df.copy()
    temp["distance"] = temp["title_asvectors"]\
                       .apply(lambda vectrs2: word_mover_distance(vectrs1,vectrs2))

    result = temp.loc[lambda df: df.distance != 0]\
             .nsmallest(n,'distance')\
             .drop(['sanitized_title', 'title_asvectors','distance'], axis=1)
    
    return result

In [87]:
get_most_similar_news(data.xs(4)["sanitized_title"], data, model, n=3)

,title,subtitle,author,date,section,text,url
179,“Nunca vou entender por que deixaram Michael J...,Diretor de ‘Deixando Neverland’ explica como l...,Tom C. Avendaño,24/03/2019 15:12:49,Cultura,O que o surpreendeu em relação à enorme reaçã...,https://brasil.elpais.com/brasil/2019/03/23/cu...
194,‘Tears in Heaven’: a tragédia que inspirou uma...,Março é uma época amarga para Eric Clapton. No...,Sara Navas,30/03/2019 14:24:02,Cultura,Quase uma década após a tragédia Lory del San...,https://brasil.elpais.com/brasil/2019/03/26/cu...
45,O trágico destino de Tom Ballard,Filho da alpinista Alison Hargreaves morta no...,Óscar Gogorza,03/03/2019 16:15:35,Esportes,Tom Ballard perdeu a mãe Alison Hargreaves a...,https://brasil.elpais.com/brasil/2019/03/02/de...
